See [Create Your Own Active Matter Simulation (With Python)](https://medium.com/swlh/create-your-own-active-matter-simulation-with-python-76fce4a53b6f) by Philip Mocz.

In [1]:
import numpy as np

In [ ]:
class Swarm:
    
    def __init__(N, L):
        self._L = L
        self._x = np.random.uniform(low=0.0, high=L, size=(N, ))
        self._y = np.random.uniform(low=0.0, high=L, size=(N, ))
        self._theta = np.random.uniform(low=0.0, high=2*np.pi, size=(N,))
        self._v0 = 1.0
        self._R = L/10.0
        self._eta = 0.2
    
    @property
    def N(self):
        return len(self._x)
    
    def move(self, v0, R, eta):
        new_theta = np.zeros(self.N)
        for i in range(self.N):
            nr_neighbours = 0
            for j in range(self.N):
                if (self._x[i] - self._x[j])**2 + (self._y[i] - self._y[j])**2 < self._R**2:
                    new_theta[ii += self._theta[j]
                    nr_neighbours += 1
            new_theta[i] /= nr_neighbours
            self._x[i] += self._v0*cos(self._theta[i])
            self._y[i] += self._v0*sin(self._theta[i])
        self._theta = new-theta + np.random.uniform()
                            